# Week 4 Project: Investor Risk Preference (Optimized)

## Goal
This project explicitly models **Risk Aversion ($\lambda$)** using Convex Optimization.

We will:
1.  Download historical data for 3 assets.
2.  Define the Mean-Variance Utility Function: $U = E[R] - \lambda \sigma^2$
3.  Use **CVXPY** to mathematically find the optimal portfolio for varying levels of $\lambda$.
4.  Visualize how asset allocation shifts from Aggressive to Conservative.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Import Data
TICKERS = ["GAIL.NS", "SILVERBEES.NS", "TATAPOWER.NS"]
print(f"Downloading data for: {TICKERS}")

# Download adjusted close prices
df = yf.download(TICKERS, period="3y")['Close']
df.dropna(inplace=True)
df.head()

In [ ]:
# 2. Calculate Annualized Returns and Covariance
log_returns = np.log(df / df.shift(1)).dropna()

TRADING_DAYS = 252
mu = log_returns.mean() * TRADING_DAYS
Sigma = log_returns.cov() * TRADING_DAYS

print("Expected Annual Returns (mu):\n", mu)
print("\nAnnualized Covariance Matrix (Sigma):\n", Sigma)

## 3. Optimization using CVXPY
We solve for weights $w$ that maximize:
$$ \text{maximize} \quad \mu^T w - \lambda (w^T \Sigma w) $$

Subject to:
$$ \sum w_i = 1 $$
$$ w_i \ge 0 \quad (\text{Long Only}) $$

In [ ]:
n_assets = len(TICKERS)
w = cp.Variable(n_assets)
lambda_param = cp.Parameter(nonneg=True)  # Risk aversion parameter

# Determine the risk term (Variance)
risk = cp.quad_form(w, Sigma.values)
expected_return = mu.values @ w

# Objective: Maximise Utility (Return - lambda * Variance)
# Note: Some formulations use lambda/2, but the behavior is identical, just scaling.
objective = cp.Maximize(expected_return - lambda_param * risk)

constraints = [
    cp.sum(w) == 1,
    w >= 0
]

problem = cp.Problem(objective, constraints)

In [ ]:
# 4. Iterate over Lambda values
lambdas = np.logspace(-2, 3, 50) # Use logspace to cover a wider range of behaviors (0.01 to 1000)
results = []

for lam_val in lambdas:
    lambda_param.value = lam_val
    try:
        problem.solve()
        if w.value is not None:
            # Capture results
            p_ret = expected_return.value
            p_vol = np.sqrt(risk.value)
            
            row = {
                'Lambda': lam_val,
                'Return': p_ret,
                'Volatility': p_vol
            }
            for i, ticker in enumerate(TICKERS):
                row[ticker] = w.value[i]
                
            results.append(row)
            
    except cp.SolverError:
        pass

optimal_df = pd.DataFrame(results)
optimal_df.head()

## 5. Visualizations

In [ ]:
# Plot 1: Asset Allocation Stacked Area Chart
plt.figure(figsize=(10, 6))
plt.stackplot(optimal_df['Lambda'], 
              [optimal_df[t] for t in TICKERS], 
              labels=TICKERS, alpha=0.8)
plt.xscale('log') # Log scale helps see the transition better
plt.xlabel("Risk Aversion $\lambda$ (Log Scale)")
plt.ylabel("Portfolio Weight")
plt.title("How Optimal Allocation Changes with Risk Aversion")
plt.legend(loc='upper right')
plt.margins(0, 0)
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Plot 2: Efficient Frontier Mapping
plt.figure(figsize=(10, 6))

# Plot the path of optimal portfolios
plt.plot(optimal_df['Volatility'], optimal_df['Return'], 
         'b-o', markersize=5, label='Optimal Portfolios (Varying $\lambda$)')

# Mark Max Return (Low Lambda)
plt.scatter(optimal_df.iloc[0]['Volatility'], optimal_df.iloc[0]['Return'], 
            color='red', s=100, label='Risk Seeking (Low $\lambda$)')

# Mark Min Risk (High Lambda)
plt.scatter(optimal_df.iloc[-1]['Volatility'], optimal_df.iloc[-1]['Return'], 
            color='green', s=100, label='Risk Averse (High $\lambda$)')

plt.xlabel("Annualized Volatility (Risk)")
plt.ylabel("Annualized Expected Return")
plt.title("The Efficient Frontier Traced by Utility Maximization")
plt.legend()
plt.grid(True)
plt.show()